In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

### Compute Growth Truth Data from Regression Results

In [61]:
df = pd.read_csv("diff_data/cems_diffs_isorto.csv", index_col='DATE_UTC')

In [62]:
# Remove columns with damage data 
df = df[df.columns[~df.columns.str.endswith('ap2')]]
df = df[df.columns[~df.columns.str.endswith('dam')]]
df = df[df.columns[~df.columns.str.endswith('eas')]]

In [63]:
# calculate MEF (assumed to be emissions / generation)
df['co2_mef'] = df['co2_kg']/df['gload_mwh']
df['so2_mef'] = df['so2_kg']/df['gload_mwh']
df['nox_mef'] = df['nox_kg']/df['gload_mwh']
df['pm25_mef'] = df['pm25_kg']/df['gload_mwh']
df = df[df.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
df = df.dropna() 

In [64]:
df.describe()

,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,co2_mef,so2_mef,nox_mef,pm25_mef
count,736057.000000,736057.000000,736057.000000,736057.000000,7.360570e+05,7.360570e+05,736057.000000,736057.000000,736057.000000
mean,0.114730,-0.439396,-0.118684,0.003811,5.577672e+01,5.912107e+02,0.649025,0.540102,0.069971
std,1467.368385,3195.501563,1085.100576,187.393820,9.780475e+05,3.870106e+04,283.239321,124.029999,11.022804
min,-10404.730000,-128718.852700,-20854.566238,-6594.509202,-7.671466e+06,-1.800064e+07,-176188.604575,-48955.437468,-6822.031204
25%,-571.530000,-372.910230,-232.132510,-34.473030,-3.201238e+05,4.184159e+02,0.001890,0.039139,0.038689
50%,-1.660000,-0.021319,-3.577480,0.000000,8.672689e+02,5.570117e+02,0.402684,0.344181,0.058396
75%,564.370000,335.336937,214.166184,33.112253,3.052859e+05,7.381082e+02,1.511817,0.800240,0.094521
max,10664.380000,72387.984792,26475.954120,2430.076459,7.822484e+06,2.451540e+07,60355.442914,69533.022803,5624.547003


### Comparison #1: Emissions Intensity from CEMS

Based on Thomas' emissions factor calculation in the simple dispatch model

In [90]:
# NOTE: only looking at states in PJM in 2007

df_raw = pd.DataFrame()

units = ['SO2_MASS (lbs)']
no_units = ['SO2_MASS']

cols_no_units = ['OP_DATE','OP_HOUR','GLOAD', 'SO2_MASS', 'NOX_MASS', 'CO2_MASS', 'HEAT_INPUT']
cols_units = ['OP_DATE','OP_HOUR','GLOAD (MW)', 'SO2_MASS (lbs)', 'NOX_MASS (lbs)', 'CO2_MASS (tons)', 'HEAT_INPUT (mmBtu)']

pjm_states = ['de', 'in', 'ky', 'md', 'mi', 'nj', 'nc', 'oh', 'tn', 'wv', 'il', 'pa', 'va', 'dc']
for state in pjm_states:
    for m in ['01','02','03','04','05','06','07','08','09','10','11', '12']:
        df_raw_add = pd.read_csv("cems_data/2007" + state + m + ".csv")
        if no_units in df_raw_add.columns.values: 
            cols = cols_no_units 
        else: 
            cols = cols_units 
        df_raw_add = df_raw_add[cols].dropna()
        df_raw_add.columns=['date','hour','mwh', 'so2_tot', 'nox_tot', 'co2_tot', 'mmbtu']
        df_raw = pd.concat([df_raw, df_raw_add], sort=False)

In [78]:
# Convert emissions to kg
df_raw.co2_tot = df_raw.co2_tot * 907.185 #tons to kg
df_raw.so2_tot = df_raw.so2_tot * 0.454 #lbs to kg
df_raw.nox_tot = df_raw.nox_tot * 0.454 #lbs to kg

# Calculate emission rates 
df_raw['co2'] = df_raw.co2_tot / df_raw.mwh
df_raw['so2'] = df_raw.so2_tot / df_raw.mwh
df_raw['nox'] = df_raw.nox_tot / df_raw.mwh

df_raw.replace([np.inf, -np.inf], np.nan, inplace=True)

In [79]:
# Aggregate over month to compare 
df_raw['date'] = pd.to_datetime(df_raw['date'])
df_raw['month'] = pd.DatetimeIndex(df_raw.date).month
group_by = df_raw.groupby(df_raw['month']).median()
group_by

# SO2 MEF is really high... 

,hour,mwh,so2_tot,nox_tot,co2_tot,mmbtu,co2,so2,nox,month
month,,,,,,,,,,
1,12,166.0,645.088600,231.442390,146773.007557,1605.5000,885.312388,4.203694,1.630880,1
2,12,163.0,668.061000,230.314200,142147.724835,1570.3000,878.961467,4.319985,1.608863,2
3,12,158.0,640.775600,225.786685,138393.793305,1531.9000,888.590525,4.173024,1.603628,3
4,12,153.0,614.125800,200.145900,132786.482820,1478.2000,887.794786,4.109155,1.485157,4
5,12,151.0,548.472406,130.933600,129183.144000,1470.5000,893.844044,3.969390,0.960062,5
6,12,148.0,544.800000,129.942972,126846.235440,1461.9000,893.318783,3.750738,0.953003,6
7,12,148.0,511.431000,120.627800,124284.345000,1468.3875,891.021561,3.537423,0.894590,7
8,13,142.0,422.174600,105.328000,108335.125515,1431.3000,869.524393,2.701763,0.714238,8
9,12,148.0,499.241100,124.078200,125650.565610,1469.1000,890.293820,3.389628,0.867140,9


In [101]:
# Compare this to 2007 regression MEFs for PJM
df_pjm = df.copy()
df_pjm['year'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).year
df_pjm['month'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).month
df_pjm = df_pjm.loc[df_pjm['isorto'] == 'PJM']
df_pjm_2017 = df_pjm.loc[df_pjm['year']==2007]

group_by = df_pjm_2017.groupby(df_pjm_2017['month']).median()
group_by

,so2_mef,nox_mef,pm25_mef,co2_mef,hour,year
month,,,,,,
1,5.115472,1.838057,0.245314,761.544918,11.5,2007.0
2,4.237025,1.563404,0.216503,718.546044,11.5,2007.0
3,4.819943,1.831405,0.231835,741.910815,11.5,2007.0
4,4.331473,1.444534,0.216914,718.124280,11.5,2007.0
5,4.570366,0.854189,0.261671,753.801370,11.5,2007.0
6,4.186646,0.796327,0.222236,733.018195,11.5,2007.0
7,3.556162,0.779180,0.195904,683.775437,11.5,2007.0
8,2.541565,0.698884,0.166140,628.340860,11.5,2007.0
9,3.706542,0.795703,0.195257,692.551026,11.5,2007.0


### Comparison #2: PJM's Published MEFs 

In [97]:
df_pjm = df_pjm.loc['2014-01-01':'2018-12-31']

# Keep only MEF cols 
df_pjm = df_pjm[mef_columns]

# convert to lbs/MWh
LBS_CONVERSION = 2.20462
df_pjm = df_pjm.applymap(lambda x: x * LBS_CONVERSION)

# Aggregate over month
df_pjm['year'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).year
df_pjm['month'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).month
group_by = df_pjm.groupby([(df_pjm['year']),(df_pjm['month'])]).median()

group_by

co2_mef   so2_mef   nox_mef  pm25_mef  year  month
year month                                                        
2014 1      3219.189041  6.774380  3.265614  0.415664  2014      1
     2      2938.429175  5.549599  2.405713  0.373701  2014      2
     3      3104.523402  6.494937  2.987428  0.399396  2014      3
     4      3326.249286  7.740751  3.165194  0.457553  2014      4
     5      3246.873666  6.529450  2.890527  0.416741  2014      5
     6      3277.870727  6.522550  2.706178  0.420024  2014      6
     7      3349.191296  6.154395  2.680979  0.443650  2014      7
     8      3383.530532  6.390969  2.792308  0.429852  2014      8
     9      3359.338985  6.207650  2.781730  0.448017  2014      9
     10     3406.466779  8.325412  3.823944  0.469397  2014     10
     11     3194.171211  8.571023  3.523845  0.434699  2014     11
     12     3228.067763  7.226979  3.049291  0.434523  2014     12
2015 1      3297.926421  7.339368  2.864309  0.464048  2015      1
     2      3190.777645  6.701805  3.258052  0.419746  2015      2
     3      3413.904134  7.779811  3.487483  0.465669  2015      3
     4      3405.665909  7.683757  3.781630  0.469131  2015      4
     5      3306.267956  5.716383  2.513193  0.422859  2015      5
     6      3338.837382  5.298036  2.587855  0.427690  2015      6
     7      3361.941719  5.353828  2.612321  0.423908  2015      7
     8      3375.084593  5.224183  2.443546  0.425592  2015      8
     9      3418.312613  5.278283  2.492972  0.431680  2015      9
     10     3352.752893  5.140281  3.060975  0.450641  2015     10
     11     3275.515095  5.149555  3.043151  0.449661  2015     11
     12     3229.012175  4.700892  2.725925  0.433141  2015     12
2016 1      3357.974315  5.502474  3.005398  0.441348  2016      1
     2      3278.681469  4.676535  2.734256  0.422086  2016      2
     3      3216.105219  4.477110  2.801360  0.414040  2016      3
     4      3437.687476  4.854822  3.189629  0.427794  2016      4
     5      3233.968900  4.151084  2.187498  0.403926  2016      5
     6      3184.532390  3.442670  1.999524  0.384976  2016      6
     7      3237.546434  3.246922  2.212014  0.378291  2016      7
     8      3261.817969  3.376718  2.024978  0.375017  2016      8
     9      3234.626343  3.458868  1.806256  0.398653  2016      9
     10     3286.797664  3.833871  2.542126  0.414622  2016     10
     11     3312.764907  3.952849  2.618324  0.427514  2016     11
     12     2985.902225  3.013709  2.444143  0.366359  2016     12
2017 1      3010.724795  2.521373  1.808584  0.367529  2017      1
     2      3058.555370  3.435440  1.979907  0.393103  2017      2
     3      2951.184269  2.484299  1.665212  0.351199  2017      3
     4      3139.262192  3.163371  1.712155  0.413233  2017      4
     5      3177.367156  3.009105  1.837995  0.391336  2017      5
     6      3106.278436  2.724755  1.547008  0.370582  2017      6
     7      3129.113128  2.932243  1.724447  0.375220  2017      7
     8      3203.823821  3.346224  1.581395  0.417274  2017      8
     9      3036.410891  2.722819  1.690988  0.367975  2017      9
     10     3140.900434  2.814165  2.030816  0.378720  2017     10
     11     3007.860695  3.275033  1.642798  0.350246  2017     11
     12     3050.223330  3.611559  1.669707  0.353862  2017     12

### Incorporate intercept value into ground truth data

In [92]:
# Use monthTOD data 
intercept_df = pd.read_csv("isorto_monthTOD.csv")

rows = ['year', 'hour', 'isorto', 'so2_kg-int', 'nox_kg-int', 'pm25_kg-int', 'co2_kg-int']
intercept_df = intercept_df[rows]

# Rename intercept values mef in order to merge
intercept_df.rename(columns={'so2_kg-int': 'so2_mef', 'nox_kg-int': 'nox_mef', 'pm25_kg-int': 'pm25_mef', 'co2_kg-int': 'co2_mef'}, inplace=True)

# Add year and hour into existing df 
df = df.reset_index()
rows = ['DATE_UTC', 'isorto', 'so2_mef', 'nox_mef', 'pm25_mef', 'co2_mef']
df = df[rows]
df['hour'] = pd.DatetimeIndex(df.DATE_UTC).hour
df['year'] = pd.DatetimeIndex(df.DATE_UTC).year

In [93]:
# Combine the two dataframes 
include_int_df = pd.merge(df, intercept_df, on=['hour', 'year', 'isorto'])
include_int_df

,DATE_UTC,isorto,so2_mef_x,nox_mef_x,pm25_mef_x,co2_mef_x,hour,year,so2_mef_y,nox_mef_y,pm25_mef_y,co2_mef_y
0,2006-01-01 04:00:00,CAISO,0.001961,0.095859,0.018454,414.653332,4,2006,-0.021749,0.109879,-0.797304,-3955.376913
1,2006-01-01 04:00:00,CAISO,0.001961,0.095859,0.018454,414.653332,4,2006,-0.025431,-16.348720,0.146704,-4738.195511
2,2006-01-01 04:00:00,CAISO,0.001961,0.095859,0.018454,414.653332,4,2006,-0.057177,-26.767284,-4.885902,-11562.777657
3,2006-01-01 04:00:00,CAISO,0.001961,0.095859,0.018454,414.653332,4,2006,-0.017067,2.104880,-1.042978,-3505.506532
4,2006-01-01 04:00:00,CAISO,0.001961,0.095859,0.018454,414.653332,4,2006,0.021364,-8.244744,1.263680,6747.131897
...,...,...,...,...,...,...,...,...,...,...,...,...
8832679,2017-12-31 23:00:00,SPP,0.603420,0.846141,0.088859,762.958625,23,2017,-264.853105,317.774965,11.570188,204476.949050
8832680,2017-12-31 23:00:00,SPP,0.603420,0.846141,0.088859,762.958625,23,2017,258.022900,-106.194461,9.554333,64207.723242
8832681,2017-12-31 23:00:00,SPP,0.603420,0.846141,0.088859,762.958625,23,2017,-46.889011,-24.403246,2.078942,35472.387540
8832682,2017-12-31 23:00:00,SPP,0.603420,0.846141,0.088859,762.958625,23,2017,15.232451,13.818577,3.725843,15611.626739


In [94]:
include_int_df['so2_mef'] = include_int_df.so2_mef_x - include_int_df.so2_mef_y
include_int_df['nox_mef'] = include_int_df.nox_mef_x - include_int_df.nox_mef_y
include_int_df['pm25_mef'] = include_int_df.pm25_mef_x - include_int_df.pm25_mef_y
include_int_df['co2_mef'] = include_int_df.co2_mef_x - include_int_df.co2_mef_y


In [95]:
rows = ['DATE_UTC', 'isorto', 'so2_mef', 'nox_mef', 'pm25_mef', 'co2_mef']

include_int_df = include_int_df[rows]

In [100]:
# Aggregate over year and month
group_by = include_int_df.groupby([(include_int_df['year']),(include_int_df['month'])]).median()
group_by

so2_mef   nox_mef  pm25_mef      co2_mef
year month                                           
2006 1      2.439719  2.411628 -0.042603  1058.805289
     2      2.117388  2.328841 -0.043684  1017.252075
     3      1.951569  2.301104 -0.053252   991.152714
     4      2.101247  2.218242 -0.033915   956.947081
     5      2.101152  2.195926 -0.045825   930.744370
...              ...       ...       ...          ...
2017 8      0.003988 -4.232927  0.052299  -500.563206
     9      0.004812 -4.244946  0.052288  -531.805482
     10     0.004502 -4.058023  0.052569  -533.281839
     11     0.004610 -4.248681  0.052323  -563.596432
     12     0.007728 -4.199429  0.052645  -537.535045

[144 rows x 4 columns]

### Why are MEFs negative? 

A negative MEF means that either: 
- Generation decreased while emissions increased 
    - is there a situation where this can happen?
- Generation increased while emissions decreased 
    - perhaps generation shifted from coal to gas 

In [85]:
# Keep only negative MEFs
mef_columns = ['co2_mef', 'so2_mef', 'nox_mef', 'pm25_mef']
df_pjm_neg = df_pjm[(df_pjm[mef_columns]<0).any(axis='columns')]

In [86]:
# Percent of rows with negative MEFs
len(df_pjm_neg.index) / len(df_pjm.index)

0.23205559675241952

In [87]:
# Percent where generation increased while emission decreased 
df_pjm_neg.gload_mwh[df_pjm_neg.gload_mwh > 0].count() / df_pjm_neg.gload_mwh.count()

0.4852308574706051

##### Are MEFs negative because generation shifted from coal to gas? 

In [71]:
# Get data from PJM data miner 
df_pjm_gen = pd.read_csv("pjm_data/pjm_gen.csv", index_col='datetime_beginning_ept')
df_pjm_gen.index = pd.to_datetime(df_pjm_gen.index)
del df_pjm_gen['datetime_beginning_utc']
del df_pjm_gen['is_renewable']

# Keep only fossil fuel rows 
fossil_fuel = ['Coal', 'Gas', 'Oil']
df_pjm_gen = df_pjm_gen.loc[df_pjm_gen['fuel_type'].isin(fossil_fuel)]
df_pjm_gen.head()

,fuel_type,mw,fuel_percentage_of_total
datetime_beginning_ept,,,
2017-01-01 00:00:00,Coal,33866.5,0.41
2017-01-01 00:00:00,Gas,10516.4,0.13
2017-01-01 00:00:00,Oil,175.7,0.00
2017-01-01 01:00:00,Coal,33164.4,0.41
2017-01-01 01:00:00,Gas,10203.0,0.13


In [88]:
# Get dates where MEFs are negative and generation increased 
df_neg_increased_gen = df_pjm_neg[df_pjm_neg.gload_mwh > 0]
neg_dates = df_neg_increased_gen.index.values
yesterday = lambda x: pd.to_datetime(x) - timedelta(hours=1) 
yesterday_arr = yesterday(neg_dates)
yesterday_arr = np.delete(yesterday_arr, 0) # remove first elem as it's in 2016

# Show dates with negative index 
gen_mix_df = df_pjm_gen[df_pjm_gen.index.isin(df_neg_increased_gen.index) | df_pjm_gen.index.isin(yesterday_arr)]

# save as csv 
gen_mix_df.to_csv('gen_mix.csv')

gen_mix_df

,mw
datetime_beginning_ept,
2017-01-01 07:00:00,43624.7
2017-01-01 08:00:00,44438.6
2017-01-01 15:00:00,42948.3
2017-01-01 16:00:00,44396.1
2017-01-02 10:00:00,51433.3
...,...
2017-12-30 06:00:00,62673.0
2017-12-30 07:00:00,63525.1
2017-12-30 08:00:00,65587.5
